In [1]:
import pandas as pd
import numpy as np
import gc

import os
import sys
sys.path.append("../")


from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import StandardScaler


import Tool.utils as utils
import Tool.config as config
from Feature import one_hot_feature

In [2]:
data = pd.read_pickle(config.data_prefix_path + 'data.pkl')

In [3]:
train = data[data.is_trade.notnull()]
test = data[data.is_trade.isnull()]
del data
gc.collect()

0

## LR线下验证

尝试过在one_hot特征上再加入原始特征，讲道理效果会更好   
但是模型跑不出来(非稀疏矩阵LR速度太慢,也有可能是我忘了scaler....)
***

In [4]:
X_train = train[train.day < 24]
X_val = train[train.day == 24]

del train,test
gc.collect()

0

In [5]:
train_X,val_X = one_hot_feature.get_one_hot_data('val')
train_Y = X_train['is_trade'].values.ravel()
val_Y = X_val['is_trade'].values.ravel()

del X_train,X_val
gc.collect()

获取类别特征one_hot线下验证数据
训练集长度: 420627
验证集长度：57405
获取xgboost特征one_hot线下验证数据
训练集长度: 420627
测试集长度：57405


6

In [19]:
lr = LogisticRegression(n_jobs=6,C=0.0005)
lr.fit(train_X, train_Y)
    
y_train_pred = lr.predict_proba(train_X)[:,1]
train_log = log_loss(train_Y,y_train_pred)
    
y_val_pred = lr.predict_proba(val_X)[:,1]
val_log = log_loss(val_Y,y_val_pred)
    
print(train_log)
print(val_log)
print(roc_auc_score(val_Y,y_val_pred))

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(self.n_jobs))


0.0797218752052
0.0799758314139
0.715859852087


## LR线上提交

In [4]:
train_X,test_X = one_hot_feature.get_one_hot_data('train')
train_Y = train['is_trade'].values.ravel()

del train
gc.collect()

获取类别特征one_hot线上提交数据
训练集长度: 478032
测试集长度：42888
获取xgboost特征one_hot线上提交数据
训练集长度: 478032
测试集长度：42888


24

In [5]:
lr = LogisticRegression(n_jobs=6,C=0.0005)
lr.fit(train_X, train_Y)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=0.0005, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=6, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [6]:
test['predicted_score'] = lr.predict_proba(test_X)[:,1]

In [7]:
test[['instance_id', 'predicted_score']].to_csv(config.data_prefix_path + 'sub0421_xgboost_LR.txt',sep=" ",index=False)